##XAI Demo - Agent + SHAP (XAI Tool)

1. Building the simple Agent making a prediction (loan approval).

2. SHAP generating an explanation for that prediction. --> SHAP (SHapley Additive exPlanations): Uses game-theoretic Shapley values to attribute a prediction to features fairly; provides local and global explanations. Which helps to interpret which features had the great influence to predict the final result.

3. The Agent giving a reasoning-style explanation based on SHAP values.

# Install dependencies

In [ ]:
!pip install shap scikit-learn

# Build a Simple Dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Create a simple dataset
data = pd.DataFrame({
    "credit_score": np.random.randint(300, 850, 500),
    "income": np.random.randint(20000, 150000, 500),
    "loan_amount": np.random.randint(1000, 50000, 500),
    "existing_loans": np.random.randint(0, 5, 500),
})

# Target: Loan Approved (1) / Rejected (0)
data["approved"] = (
    (data["credit_score"] > 600) &
    (data["income"] > 40000) &
    (data["existing_loans"] < 2)
).astype(int)

X = data.drop("approved", axis=1)
y = data["approved"]


In [ ]:
data

,credit_score,income,loan_amount,existing_loans,approved
0,351,132682,28509,1,0
1,696,66673,10817,2,0
2,693,57477,42117,2,0
3,573,39397,36399,0,0
4,442,34527,24487,4,0
...,...,...,...,...,...
495,457,70979,33543,1,0
496,450,34952,8734,4,0
497,321,42187,42754,0,0
498,365,138810,49961,4,0


# Train a Model - Here I pick the RandomForest as it's tree based

In [ ]:
model = RandomForestClassifier()
model.fit(X, y)


RandomForestClassifier()

# Select a New Applicant (Agent Input)

In [ ]:
new_applicant = pd.DataFrame([{
    "credit_score": 540,
    "income": 35000,
    "loan_amount": 20000,
    "existing_loans": 3
}])

model.predict(new_applicant)[0]


np.int64(0)

# SHAP Explanation - The below results showing which features pulled the decision towards rejection.

In [ ]:
prediction = model.predict(new_applicant)[0]

import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(new_applicant)

# Use the correct SHAP vector
'''
1 → one applicant/sample

4 → four features

2 → two classes (0=rejected, 1=approved)

Here class 0 SHAP shows  -> “How much each feature supports the rejection prediction.”
'''
shap_for_applicant = shap_values[0, :, 0]   # shap_values[sample_index, feature_index, class_index]

contrib = pd.DataFrame({
    "feature": X.columns,
    "shap_value": shap_for_applicant
}).sort_values("shap_value", key=abs, ascending=False)

contrib
# agent_reasoning(new_applicant, contrib, prediction)


,feature,shap_value
0,credit_score,0.057406
3,existing_loans,0.055937
1,income,0.048864
2,loan_amount,0.000074


# Agent Reasoning Layer

simulate how an Agent would explain its reasoning using SHAP.

In [ ]:
def agent_reasoning(applicant, shap_table, prediction):
    print("\n===== Agent Decision =====")
    print("Loan Prediction:", "Approved" if prediction == 1 else "Rejected")

    print("\n===== XAI Explanation (SHAP) =====")
    for _, row in shap_table.iterrows():
        print(f"{row['feature']}: contribution = {round(row['shap_value'], 2)}")

    print("\n===== Agent Summary =====")
    top_feature = shap_table.iloc[0]
    print(f"The loan was likely {'approved' if prediction == 1 else 'rejected'} because:")
    print(f"- '{top_feature['feature']}' had the highest impact ({round(top_feature['shap_value'], 2)}).")

    print("\n===== Counterfactual Suggestion =====")
    if applicant['credit_score'].iloc[0] < 600:
        print("- Increasing credit score above 600 would likely change the prediction.")
    if applicant['existing_loans'].iloc[0] > 2:
        print("- Reducing existing loans to less than 2 increases approval chances.")
    if applicant['income'].iloc[0] < 50000:
        print("- Increasing income above 50k improves approval probability.")


# Kick the agent to run

In [ ]:
prediction = model.predict(new_applicant)[0]
agent_reasoning(new_applicant, contrib, prediction)



===== Agent Decision =====
Loan Prediction: Rejected

===== XAI Explanation (SHAP) =====
credit_score: contribution = 0.06
existing_loans: contribution = 0.06
income: contribution = 0.05
loan_amount: contribution = 0.0

===== Agent Summary =====
The loan was likely rejected because:
- 'credit_score' had the highest impact (0.06).

===== Counterfactual Suggestion =====
- Increasing credit score above 600 would likely change the prediction.
- Reducing existing loans to less than 2 increases approval chances.
- Increasing income above 50k improves approval probability.
